# SingularityCE example

This Notebook runs on the SDumont.

References:

- User guide: <https://docs.sylabs.io/guides/4.2/user-guide/>
- <https://github.com/sylabs/singularity/releases/tag/v4.2.1>
- <https://github.com/LucasFernando-aes/SDumont-UserPOV/blob/main/singularity.md>
- <https://guiesbibtic.upf.edu/recerca/hpc/singularity>
- <https://pawseysc.github.io/hpc-container-training/13-singularity-intro/>
- <https://epcced.github.io/2020-12-08-Containers-Online/aio/index.html>
- <https://singularity-tutorial.github.io/02-basic-usage/>
- <https://hsf-training.github.io/hsf-training-singularity-webpage/>
- <https://catalog.ngc.nvidia.com/orgs/nvidia/containers/pytorch/tags>

Check for related modules:

In [2]:
! module avail 2>&1 | grep -i -E "docker|apptainer|singularity|udocker"

   parabricks/2.5.0_singularity_sequana
   parabricks/3.0_singularity_sequana                            (D)


Check the available Singularity version:

In [3]:
! singularity --version

singularity-ce version 4.2.1-1


Download example container:

In [4]:
! singularity pull --force library://godlovedc/funny/lolcow

INFO:    Using cached image


In [5]:
! ls lolcow_latest.sif

lolcow_latest.sif


The following cell ran in a terminal, starting a container and running an interactive shell inside it:

Executing a command within the container, via the host's command line:

In [6]:
! singularity exec lolcow_latest.sif cat /etc/os-release

NAME="Ubuntu"
VERSION="16.04.5 LTS (Xenial Xerus)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 16.04.5 LTS"
VERSION_ID="16.04"
HOME_URL="http://www.ubuntu.com/"
SUPPORT_URL="http://help.ubuntu.com/"
BUG_REPORT_URL="http://bugs.launchpad.net/ubuntu/"
VERSION_CODENAME=xenial
UBUNTU_CODENAME=xenial


In [7]:
! singularity exec lolcow_latest.sif cowsay moo

 _____
< moo >
 -----
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||


## Another example, downloading the PyTorch container

<https://catalog.ngc.nvidia.com/orgs/nvidia/containers/pytorch/tags>

nvcr.io/nvidia/pytorch:25.04-py3

run on terminal:

    export SINGULARITY_DOCKER_USERNAME='$oauthtoken'
    export SINGULARITY_DOCKER_PASSWORD=<key>
    singularity pull pytorch25.04.sif docker://nvcr.io/nvidia/pytorch:25.04-py3

It is not possible to download for this container, a timeout occurs on the login node, and on an interactive node an error message appears when authenticating in the Nvidia repo. The interactive execution node is configured using:

    salloc --partition=sequana_gpu_shared --nodes=1 --ntasks=2 --time=30 --job-name test01
    ssh sdumontxxxx
    ...
    exit
    exit

The solution is to download to the local machine using Docker, save as .tar, and transfer to SDumont using scp or syncthing. On the local machine:

    docker login nvcr.io    # USERNAME&PASSWORD
    docker pull nvcr.io/nvidia/pytorch:22.03-py3
    docker save nvcr.io/nvidia/pytorch:22.03-py3  > local.tar
    transfer (to the image/ directory)

On the login node it is not possible to convert the image to SIF format:

In [1]:
%%bash
cd ../image
ls
time singularity build local_tar.sif docker-archive://local.tar

local.tar


INFO:    Starting build...
Getting image source signatures
Copying blob sha256:867d0767a47c392f80acb51572851923d6d3e55289828b0cd84a96ba342660c7
Copying blob sha256:01e996931197162c4d65bfc6867f243df01e5f70f877c1c9beccd6978297e643
Copying blob sha256:2ff0ade8d3c97b175cf6be4658d30fc655fa86b0198224c8f5b18e39cdd97e5f
Copying blob sha256:fec6965e7a6b67b61a1337a169d91dcbb93ebeee67bb5d38a241f7fa4fa048ea
Copying blob sha256:83cdade3c9b5e8313ab6d74bcfcc0492a4ae4b9fbf3d09a35ab52abaaf492d9d
Copying blob sha256:a060c5cefec7f091c555e77e46d0130a49dfc39023365e216c154613603f456a
Copying blob sha256:2df8c0a32afe2d088aafe59dc368643b14ddd15f9673cac769548c28ec384e04
Copying blob sha256:899455397741dac157e85ecf485ec909e52ce1ada5d37853c93544c13e7f5f81
Copying blob sha256:2f175b794573ab5e137bd2f5f52a2c1716bb5287ca8755ea8ad166fa9b1ee898
Copying blob sha256:85f49f4e6923e73ae6c3ab79d4b7f1f8156141952f3f69535ca49f6a543f9e24
Copying blob sha256:6a1014d46250ad84e998df518cecf334cf4dc3255d8da2107f5b14adb0137638
Copyin

CalledProcessError: Command 'b'cd ../image\nls\nsingularity build local_tar.sif docker-archive://local.tar\n'' returned non-zero exit status 255.

You need to copy the .tar.gz to scratch, start an interactive shell, and then convert to SIF:

In [2]:
! cp local.tar /scratch/<user-dir>/container/

In a shell:

    salloc --partition=sequana_gpu_shared --nodes=1 --ntasks=2 --time=30 --job-name test01
    ssh sdumontxxxx
    cd ...
    singularity build pytorch22.03.sif docker-archive://local.tar
    ...

Running on the sdumont18:

In [11]:
! singularity exec --nv /scratch/ampemi/<urser>/container/pytorch22.03.sif nvidia-smi

Tue Jun 11 14:23:27 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   41C    P0    38W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000000:5E:00.0 Off |                    0 |
| N/A   

Running interactively in a shell:

    $ singularity shell --nv pytorch22.03.sif
    Singularity> nvidia-smi
    Tue Jun 11 11:15:49 2024       
    +-----------------------------------------------------------------------------+
    | NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.6     |
    |-------------------------------+----------------------+----------------------+
    | GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
    | Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
    |                               |                      |               MIG M. |
    |===============================+======================+======================|
    |   0  Tesla V100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
    | N/A   45C    P0    39W / 250W |      0MiB / 32510MiB |      0%      Default |
    |                               |                      |                  N/A |
    +-------------------------------+----------------------+----------------------+
    |   1  Tesla V100-PCIE...  Off  | 00000000:5E:00.0 Off |                    0 |
    | N/A   36C    P0    37W / 250W |      0MiB / 32510MiB |      0%      Default |
    |                               |                      |                  N/A |
    +-------------------------------+----------------------+----------------------+
    |   2  Tesla V100-PCIE...  Off  | 00000000:86:00.0 Off |                    0 |
    | N/A   35C    P0    35W / 250W |      0MiB / 32510MiB |      0%      Default |
    |                               |                      |                  N/A |
    +-------------------------------+----------------------+----------------------+
    |   3  Tesla V100-PCIE...  Off  | 00000000:AF:00.0 Off |                    0 |
    | N/A   36C    P0    35W / 250W |      0MiB / 32510MiB |      2%      Default |
    |                               |                      |                  N/A |
    +-------------------------------+----------------------+----------------------+
                                                                                   
    +-----------------------------------------------------------------------------+
    | Processes:                                                                  |
    |  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
    |        ID   ID                                                   Usage      |
    |=============================================================================|
    |  No running processes found                                                 |
    +-----------------------------------------------------------------------------+
    Singularity> 

Check PyTorch:

```python
$ singularity shell --nv pytorch22.03.sif
Singularity> python
Python 3.8.12 | packaged by conda-forge | (default, Jan 30 2022, 23:42:07) 
[GCC 9.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import torch
>>> x = torch.rand(5, 3)
>>> print(x)
tensor([[0.7518, 0.3518, 0.3545],
        [0.5821, 0.3066, 0.6131],
        [0.0897, 0.5199, 0.5225],
        [0.4321, 0.3631, 0.0693],
        [0.1293, 0.1515, 0.0600]])
>>>
>>> import torch
>>> torch.cuda.is_available()
True
```